In [ ]:
"""
Created on Mon Jul 10 18:47 2023

Use cdo to regrid the BedMachine to a lower resolution

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Transformer
import pandas as pd
import sys,os
import time
from tqdm.notebook import tqdm
from cdo import Cdo
import basal_melt_param.plume_functions as pf
import cc3d

In [ ]:
%matplotlib qt5

In [ ]:
inputpath_BedMachine='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
#inputpath_BedMachine_orig='/bettik/burgardc/DATA/ERWIN_PAPER/'
inputpath_BedMachine_orig='/bettik/burgardc/DATA/SUMMER_PAPER/raw/'

SET ALL LAND TO LAND

In [ ]:
BedMachine_orig = xr.open_dataset(inputpath_BedMachine_orig+'BedMachineAntarctica_2020-07-15_v02.nc')


In [ ]:
mask_0_2_3 = BedMachine_orig['mask'].where(BedMachine_orig['mask'] != 4, 2).where(BedMachine_orig['mask'] != 1, 2)
mask_0_1_2 = mask_0_2_3.where(mask_0_2_3 != 3, 1).astype(float)
mask_0_1_2.attrs['flag_meanings'] = 'ocean floating_ice ice_free_land_and_grounded_ice'
mask_0_1_2.attrs['flag_values'] = '[0 1 2]'
mask_0_1_2.attrs['valid_range'] = '[0 2]'
mask_0_1_2.to_dataset(name='mask_0_1_2').to_netcdf(inputpath_BedMachine_orig + 'BedMachine_v2_mask_012.nc')

SET LAND VOSTOK TO GROUNDED ICE

In [ ]:
mask_0_1_2_3 = BedMachine_orig['mask'].where(BedMachine_orig['mask'] != 4, 2).astype(float)
mask_0_1_2_3.attrs['flag_meanings'] = 'ocean ice_free_land grounded_ice floating_ice'
mask_0_1_2_3.attrs['flag_values'] = '[0 1 2 3]'
mask_0_1_2_3.attrs['valid_range'] = '[0 3]'
mask_0_1_2_3.to_dataset(name='mask_0_1_2_3').to_netcdf(inputpath_BedMachine_orig + 'BedMachine_v2_mask_0123.nc')

DOWNSCALE TO 4 KM

In [ ]:
weights_filter = np.zeros((9,9)) + 1
weights_filter[0,:] = 0.5
weights_filter[:,0] = 0.5
weights_filter[8,:] = 0.5
weights_filter[:,8] = 0.5
weights_filter[0,8] = 0.25
weights_filter[8,0] = 0.25
weights_filter[8,8] = 0.25
weights_filter[0,0] = 0.25

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

In [ ]:
mask_0_1_2_3 = xr.open_dataset(inputpath_BedMachine_orig + 'BedMachine_v2_mask_0123.nc')['mask_0_1_2_3']
ISMIP_grid_4km = xr.open_dataset(inputpath_BedMachine+'ISMIP6_AIS_4000m_grid.nc')

In [ ]:
mask_0_1_2_3

In [ ]:
mask_ocean = (mask_0_1_2_3 == 0).astype(float)
mask_rock = (mask_0_1_2_3 == 1).astype(float)
mask_ground = (mask_0_1_2_3 == 2).astype(float)
mask_isf = (mask_0_1_2_3 == 3).astype(float)

In [ ]:
corr_ocean = pf.xr_nd_corr_v2(mask_ocean, weights_filter).sel(x=ISMIP_grid_4km.x,y=ISMIP_grid_4km.y)
corr_rock = pf.xr_nd_corr_v2(mask_rock, weights_filter).sel(x=ISMIP_grid_4km.x,y=ISMIP_grid_4km.y)
corr_ground = pf.xr_nd_corr_v2(mask_ground, weights_filter).sel(x=ISMIP_grid_4km.x,y=ISMIP_grid_4km.y)
corr_isf = pf.xr_nd_corr_v2(mask_isf, weights_filter).sel(x=ISMIP_grid_4km.x,y=ISMIP_grid_4km.y)

compute concentrations

In [ ]:
corr_ocean_norm = corr_ocean/64
corr_rock_norm = corr_rock/64
corr_ground_norm = corr_ground/64
corr_isf_norm = corr_isf/64

set everything where there is a bit of ice shelf to 1

In [ ]:
ls_mask_01 = ((corr_ground_norm > 0) | (corr_rock_norm > 0)).astype(int)
ls_mask_02 = ls_mask_01.where(ls_mask_01 == 0, 2)
ls_mask_012 = ls_mask_02.where(corr_isf == 0, 1)

correct ice shelf points that are within grounded ice

In [ ]:
dx = abs(ls_mask_012.x[1] - ls_mask_012.x[0]).values.astype(int)
dy = abs(ls_mask_012.y[1] - ls_mask_012.y[0]).values.astype(int)

In [ ]:
isf_only_mask = ls_mask_012 == 1

connectivity = 4
threshold = 1

dusted = cc3d.dust(isf_only_mask.values.astype(np.int64), 
           threshold = threshold, 
           connectivity = connectivity, 
           in_place = False)

labels_out = cc3d.connected_components(dusted, 
                               connectivity = connectivity)

labelled = xr.DataArray(labels_out, 
                coords = {"y": ls_mask_012.y, "x": ls_mask_012.x}, 
                dims = ["y", "x"],
                name = "labels")

# filter that checks the point around
weights_filter = np.zeros((3,3))
weights_filter[0,1] = 1
weights_filter[1,0] = 1
weights_filter[1,2] = 1
weights_filter[2,1] = 1

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

new_mask = ls_mask_012.copy()
new_corr_ground_norm = corr_ground_norm.copy()
new_corr_isf_norm = corr_isf_norm.copy()

for conn_label in tqdm(range(1,labels_out.max()+1)):
    dom_region = isf_only_mask.where(labelled == conn_label, drop=True)
    dom_bounds_plus1 = np.array([dom_region.x.min().values - dx,dom_region.x.max().values + dx,dom_region.y.min().values - dy,dom_region.y.max().values + dy]).astype(int)
    dom_plus1_mask = isf_only_mask.sel(x=range(dom_bounds_plus1[0],dom_bounds_plus1[1]+1,dx), y=range(dom_bounds_plus1[2],dom_bounds_plus1[3]+1,dy))
    corr = pf.xr_nd_corr_v2(dom_plus1_mask, weights_filter)
    only_contour = (corr ^ dom_plus1_mask)
    neighboring_pixels = ls_mask_012.where(only_contour).where(labelled != conn_label)
    if neighboring_pixels.min() > 0:
        print('There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!')
        new_mask = new_mask.where(labelled != conn_label, 2)
        new_corr_ground_norm = new_corr_ground_norm.where(labelled != conn_label, corr_isf_norm + corr_ground_norm)
        new_corr_isf_norm = new_corr_isf_norm.where(labelled != conn_label, 0)

correct ocean points that are within ice

In [ ]:
ocean_only_mask = ls_mask_012 == 0

connectivity = 4
threshold = 1

dusted = cc3d.dust(ocean_only_mask.values.astype(np.int64), 
           threshold = threshold, 
           connectivity = connectivity, 
           in_place = False)

labels_out = cc3d.connected_components(dusted, 
                               connectivity = connectivity)

labelled = xr.DataArray(labels_out, 
                coords = {"y": ls_mask_012.y, "x": ls_mask_012.x}, 
                dims = ["y", "x"],
                name = "labels")

# filter that checks the point around
weights_filter = np.zeros((3,3))
weights_filter[0,1] = 1
weights_filter[1,0] = 1
weights_filter[1,2] = 1
weights_filter[2,1] = 1

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

new_corr_ocean_norm = corr_ocean_norm.copy()

for conn_label in tqdm(range(2,labels_out.max()+1)):
    dom_region = ocean_only_mask.where(labelled == conn_label, drop=True)
    dom_bounds_plus1 = np.array([dom_region.x.min().values - dx,dom_region.x.max().values + dx,dom_region.y.min().values - dy,dom_region.y.max().values + dy]).astype(int)
    dom_plus1_mask = ocean_only_mask.sel(x=range(dom_bounds_plus1[0],dom_bounds_plus1[1]+1,dx), y=range(dom_bounds_plus1[2],dom_bounds_plus1[3]+1,dy))
    corr = pf.xr_nd_corr_v2(dom_plus1_mask, weights_filter)
    only_contour = (corr ^ dom_plus1_mask)
    neighboring_pixels = ls_mask_012.where(only_contour).where(labelled != conn_label)
    if neighboring_pixels.min() > 0:
        print('There is no ocean around it, this cannot be an ocean point')
        #print(neighboring_pixels.min().values)
        new_mask = new_mask.where(labelled != conn_label, neighboring_pixels.min().values)
        if neighboring_pixels.min().values == 1:
            new_corr_isf_norm = new_corr_isf_norm.where(labelled != conn_label, corr_ocean_norm + corr_isf_norm)
        elif neighboring_pixels.min().values == 2:
            new_corr_ground_norm = new_corr_ground_norm.where(labelled != conn_label, corr_ocean_norm + corr_ground_norm)
        new_corr_ocean_norm = new_corr_ocean_norm.where(labelled != conn_label, 0)

Checking the sums

In [ ]:
sum_norms = new_corr_ocean_norm + new_corr_isf_norm + new_corr_ground_norm + corr_rock_norm

In [ ]:
sum_norms.min()

Writing to netcdf

In [ ]:
new_corr_ocean_norm.plot()

In [ ]:
all_masks = new_mask.to_dataset(name='mask_0_1_2')
all_masks['mask_0_1_2'].attrs['flag_meanings'] = 'only_ocean contains_floating_ice only_ice_free_land_and_grounded_ice'
all_masks['mask_0_1_2'].attrs['flag_values'] = '[0 1 2]'
#all_masks['mask_0_1_2'].attrs['valid_range'] = '[0 2]'

all_masks['ocean_conc'] = new_corr_ocean_norm
all_masks['ocean_conc'].attrs['long_name'] = 'Ocean concentration between 0 and 1'
all_masks['ocean_conc'].attrs['valid_range'] = '[0 1]'

all_masks['ground_conc'] = new_corr_ground_norm
all_masks['ground_conc'].attrs['long_name'] = 'Grounded ice concentration between 0 and 1'
#all_masks['ground_conc'].attrs['valid_range'] = '[0 1]'

all_masks['isf_conc'] = new_corr_isf_norm
all_masks['isf_conc'].attrs['long_name'] = 'Floating ice concentration between 0 and 1'
#all_masks['isf_conc'].attrs['valid_range'] = '[0 1]'

all_masks['icefree_conc'] = corr_rock_norm
all_masks['icefree_conc'].attrs['long_name'] = 'Ice-free ground concentration between 0 and 1'
#all_masks['icefree_conc'].attrs['valid_range'] = '[0 1]'

all_masks.to_netcdf(inputpath_BedMachine + 'BedMachine_v2_aggregated4km_masks_only.nc')

OTHER VARIABLES

In [ ]:
all_masks = xr.open_dataset(inputpath_BedMachine + 'BedMachine_v2_aggregated4km_masks_only.nc')

In [ ]:
weights_filter = np.zeros((9,9)) + 1
weights_filter[0,:] = 0.5
weights_filter[:,0] = 0.5
weights_filter[8,:] = 0.5
weights_filter[:,8] = 0.5
weights_filter[0,8] = 0.25
weights_filter[8,0] = 0.25
weights_filter[8,8] = 0.25
weights_filter[0,0] = 0.25

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

In [ ]:
for vvar in ['firn','surface','thickness','bed','errbed','geoid']:
    print(vvar)
    var_field = BedMachine_orig[vvar].astype(float)
    var_4km = pf.xr_nd_corr_v2(var_field, weights_filter).sel(x=ISMIP_grid_4km.x,y=ISMIP_grid_4km.y)
    all_masks[vvar] = var_4km/64
    all_masks[vvar].attrs = BedMachine_orig[vvar].attrs
    

In [ ]:
all_masks.to_netcdf(inputpath_BedMachine + 'BedMachine_v2_aggregated4km_allvars.nc')

In [ ]:
all_masks['surface'].plot()

In [ ]:
all_masks['bed'].plot()

In [ ]:
BedMachine_orig['surface'].plot()